# 회귀 실습 : 캐글 주택 가격_고급 회귀 기법

- 데이터  

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data  

- train.csv  다운로드 받아서 house_price.csv로 저장

**house_price 데이터** 
- 79개의 변수로 구성
- 미국 아이오와 주의 에임즈(Ames) 지방의 주택 가격 정보 데이터

**성능 평가 : RMSLE(Root Mean Squared Log Error)**

**전체 회귀 분석 과정**
- 데이터 전처리
- 선형 회귀 모델 학습/예측/평가
- 회귀 트리 모델 학습/에측/평가
- 회귀 모델의 예측 결과 혼합을 통한 최종 예측
- 스태킹 앙상블 모델을 통한 회귀 예측

## 데이터 전처리(Preprocessing)

### 테이터 탐색
- 결측치 확인
- 피처(변수)별 분포

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

# Target 값 : SalePrice (마지막 칼럼)

주요 피처들

- 1stFlrSF: First Floor square feet  
- 2ndFlrSF: Second floor square feet  
- GrLivArea: Above grade (ground) living area square feet      
- Bedroom: Bedrooms above grade (does NOT include basement bedrooms)  
- LotArea: Lot size in square feet      
- GarageArea: Size of garage in square feet      
- OverallQual: Rates the overall material and finish of the house  
       10	Very Excellent  
       9	Excellent  
       8	Very Good  
       7	Good  
       6	Above Average  
       5	Average  
       4	Below Average  
       3	Fair  
       2	Poor  
       1	Very Poor	  
- OverallCond: Rates the overall condition of the house  
       10	Very Excellent  
       9	Excellent  
       8	Very Good  
       7	Good  
       6	Above Average	  
       5	Average  
       4	Below Average	  
       3	Fair  
       2	Poor  
       1	Very Poor  
- YearBuilt: Original construction date  
- Neighborhood: Physical locations within Ames city limits  
   Blmngtn  Bloomington Heights  
   Blueste  Bluestem  
   BrDale   Briardale  
   .....

- RoofMatl: Roof material  
	ClyTile	Clay or Tile  
	CompShg	Standard (Composite) Shingle  
	Membran	Membrane  
	Metal	Metal  
	.....

- RoofStyle: Type of roof  
	Flat	Flat  
	Gable	Gable  
	Gambrel	Gabrel (Barn)  
	.....

**데이터 타입 확인**

**Null 컬럼 및 갯수 확인**

In [ ]:
# Null 컬럼 확인 : Null이 있는 컬럼그 그 건수를 내림차순으로 출력


In [ ]:
# Null 컬럼과 그 건수 출력


In [ ]:
# 참고 : 전체 피처들의 Null 수 확인


In [ ]:
# 참고 : 데이터 타입별 수 : 시리즈 인덱스 확인


In [ ]:
# 참고 : 시리즈 값 


### 회귀 모델 적용하기 전에 타깃(y) 값의 분포가 대칭 분포인지 확인

**타겟값 SalePrice의 분포 확인**

**SalePrice의 로그 변환**

In [ ]:
# SalePrice 로그 변환


In [ ]:
# 결과값을 로그 변환하고 다시 분포도 확인


### 결측치 처리

In [ ]:
# Null 컬럼과 그 건수 출력


In [ ]:
# Null 이 너무 많은 컬럼들과 불필요한 컬럼 삭제


In [ ]:
# Drop 하지 않는 숫자형 Null컬럼들은 평균값으로 대체


In [ ]:
# Null 값이 있는 피처명과 타입 추출


In [ ]:
# 참고 : Null의 개수 확인


### 원-핫 인코딩 변환

: 문자열값은 모두 카테고리값. 판다스의 get_dummies( )를 이용하여 원-핫 인코딩 수행

In [ ]:
# 원-핫 인코딩 수행 전/후 데이터 shape 출력


## 선형 회귀 모델의 학습/예측/평가

LinearRegression, Ridge, Lasso를 이용해 선형 계열의 회귀 모델 만들기

**RMSE 평가 함수 생성**

In [ ]:
# 단일 모델의 RMSE 값 반환
def get_rmse(model): # 학습된 모델을 받아서 예측
    pass


# 여러 모델의 RMSE 값 반환
def get_rmses(models):
    pass

### 선형 회귀 모델 학습, 예측, 평가(1)
- LinearRegression, Ridge, Lasso 회귀

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



**get_top_bottom_coef(model) 함수 정의**

피처 수가 많으므로 회귀 계수 값의 상위 10개, 하위 10개(-값으로 가장 큰 10개)의  
피처명과 회귀 계수값을 가지는 Series 반환하는 함수 생성  

In [ ]:
def get_top_bottom_coef(model):
    pass


**인자로 입력되는 여러개의 회귀 모델들에 대한 회귀계수값과 컬럼명 시각화**

In [ ]:
def visualize_coefficient(models):
    pass

In [ ]:
# 앞 예제에서 학습한 lr_reg, ridge_reg, lasso_reg 모델의 회귀 계수 시각화 


**5 폴드 교차검증으로 모델별로 RMSE와 평균 RMSE출력**

In [ ]:
from sklearn.model_selection import cross_val_score

def get_avg_rmse_cv(models):
    pass

In [ ]:
# 앞 예제에서 학습한 lr_reg, ridge_reg, lasso_reg 모델의 CV RMSE값 출력           


**각 모델들의 alpha값을 변경하면서 하이퍼 파라미터 튜닝** 

In [ ]:
from sklearn.model_selection import GridSearchCV

def get_best_params(model, params):
    pass

**최적화된 alpha값으로 재학습/예측/평가**

In [ ]:
# 앞의 최적화 alpha값으로 학습데이터로 학습, 테스트 데이터로 예측 및 평가 수행. 

# 모든 모델의 RMSE 출력

# 모든 모델의 회귀 계수 시각화 


## 선형 회귀 모델 학습/예측/평가(2)

- 피처 데이터의 분포 확인(왜도) -> 로그변환  
- 이상치 데이터 처리

### 1) 피처별 왜도(skewness) 측정

**숫자형 피처들의 왜도 계산**

In [ ]:
# 숫자 피처별 왜도 계산
from scipy.stats import skew



**왜도가 1이상이 피처들 추출**

In [ ]:
# skew 정도가 1 이상인 컬럼들만 추출. 


**왜도가 1 이상이 피처들 로그 변환**

In [ ]:
# 추출된 왜곡 정도가 높은 피처를 로그 변환


In [ ]:
# 로그 변환 후 다시 왜곡 정도를 확인해 보면


**왜도가 큰 피처들의 로그변환 후 원-핫 인코딩 적용 후 회귀모델 최적 파라미터 튜닝**

In [ ]:
# 원-핫 인코딩

# 피처/타겟 데이터 세트 다시 생성

# 피처들을 로그 변환 후 다시 최적 하이퍼 파라미터와 RMSE 출력


**최적 파라미터로 재학습/예측/평가**

In [ ]:
# 앞의 최적화 alpha값으로 학습데이터로 학습, 테스트 데이터로 예측 및 평가 수행. 

# 모든 모델의 RMSE 출력

# 모든 모델의 회귀 계수 시각화 


### 2) 이상치 처리

- 회귀 계수가 큰 피처들은 예측에 많은 영향을 미치는 중요 피처이므로 이상치 데이터 처리가 중요함  
- 먼저 세 개 모델 모두에서 가장 큰 회귀 계수를 가지는 GrLivArea(주거 공간 크기) 피처의 데이터 분포 확인

**GrLivArea값과 SalePrice와의 산점도**

In [ ]:
# 주택 가격 데이터가 변환되기 이전의 원본 데이터 세트에서
# GrLivArea(주거 공간 크기)와 타깃 값인 SalePrice의 관계 시각화


**이상치 데이터 삭제**

In [ ]:
# house_df_ohe에서 이상치 데이터 필터링


In [ ]:
# DataFrame의 index를 이용하여 아웃라이어 레코드 삭제 


**릿지, 라쏘 모델 최적화**

In [ ]:
# house_df_ohe 기반으로 피처/타깃 데이터 세트 다시 생성하고
# 릿지와 라쏘 모델의 최적화 수행



**이상치가 제거된 데이터세트 기반으로 최적화 된 alpha 적용하여 학습/평가**

In [ ]:
# 이상치가 제거된 데이터 세트를 기반으로 
# 다시 분할된 데이터 세트의 RMSE 수치 및 회귀 계수 시각화

# 앞의 최적화 alpha값으로 학습데이터로 학습, 테스트 데이터로 예측 및 평가 수행. 

# 모든 모델의 RMSE 출력

# 모든 모델의 회귀 계수 시각화 


## 회귀 트리 학습/예측/평가 

**XGBoost와 LightGBM 학습/예측/평가**

**XGBoost 회귀트리**

In [ ]:
# 수행 시간이 오래 걸릴 수 있는 관계로
# 하이퍼 파라미터 설정을 미리 적용한 상태로
# 5 폴드 세트에 대한 평균 RMSE 값 추출

# XGBoost 회귀 트리


**LightGBM 회귀 트리**

In [ ]:
# LightGBM 회귀 트리



**트리 회귀 모델의 피처 중요도 시각화**

In [ ]:
# 모델의 중요도 상위 20개의 피처명과 그때의 중요도값을 Series로 반환

def get_top_features(model):
    ftr_importances_values = model.feature_importances_
    ftr_importances = pd.Series(ftr_importances_values, index=X_features.columns  )
    ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
    return ftr_top20

def visualize_ftr_importances(models):
    fig, axs = plt.subplots(figsize=(24,10),nrows=1, ncols=2)
    fig.tight_layout() 
    for i_num, model in enumerate(models):
        ftr_top20 = get_top_features(model)
        axs[i_num].set_title(model.__class__.__name__+' Feature Importances', size=25)
        for label in (axs[i_num].get_xticklabels() + axs[i_num].get_yticklabels()):
            label.set_fontsize(22)
        sns.barplot(x=ftr_top20.values, y=ftr_top20.index , ax=axs[i_num])

### 회귀 모델들의 예측 결과 혼합을 통한 최종 예측

In [ ]:
# 앞에서 구한 릿지 모델(40%)과 라쏘 모델(60%) 혼합
# 개별 모델과 최종 혼합 모델의 RMSE 추출

# 최종 혼합 모델과 개별 모델의 RMSE 값 출력하는 함수 생성
def get_rmse_pred(preds):
    for key in preds.keys():
        pred_value = preds[key]
        mse = mean_squared_error(y_test , pred_value)
        rmse = np.sqrt(mse)
        print('{0} 모델의 RMSE: {1}'.format(key, rmse))

# 개별 모델의 학습

# 개별 모델 예측

# 개별 모델 예측값 혼합으로 최종 예측값 도출

#최종 혼합 모델, 개별모델의 RMSE 값 출력



In [ ]:
# XBBoost와 LightGBM 혼합 (각 50%)

# XGBRegressor()에 objective='reg:squarederror' 추가 ('reg:lineaqr'가 변경됨)


## 스태킹 모델을 통한 회귀 예측

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)
    
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        model.fit(X_tr , y_tr)       
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    return train_fold_pred, test_pred_mean

In [ ]:
# 기반 모델별로 get_stacking_base_datasets() 함수 적용해서 
# 메타 모델이 사용할 학습/테스트 피처 데이터 세트 추출

# get_stacking_base_datasets( )은 넘파이 ndarray를 인자로 사용하므로 DataFrame을 넘파이로 변환. 
X_train_n = X_train.values
X_test_n = X_test.values
y_train_n = y_train.values

# 각 개별 기반(Base)모델이 생성한 학습용/테스트용 데이터 반환. 
ridge_train, ridge_test = get_stacking_base_datasets(ridge_reg, 
                                                     X_train_n, 
                                                     y_train_n, 
                                                     X_test_n, 5)
lasso_train, lasso_test = get_stacking_base_datasets(lasso_reg,
                                                     X_train_n, 
                                                     y_train_n, 
                                                     X_test_n, 5)
xgb_train, xgb_test = get_stacking_base_datasets(xgb_reg, 
                                                 X_train_n, 
                                                 y_train_n, 
                                                 X_test_n, 5)  
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_reg, 
                                                   X_train_n, 
                                                   y_train_n, 
                                                   X_test_n, 5)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 Stacking 형태로 결합.  
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, 
                                      xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, 
                                     xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 라쏘 모델을 적용. 
meta_model_lasso = Lasso(alpha=0.0005)

#기반 모델의 예측값을 기반으로 새롭게 만들어진 학습 및 테스트용 데이터로 예측하고 RMSE 측정.
meta_model_lasso.fit(Stack_final_X_train, y_train)
final = meta_model_lasso.predict(Stack_final_X_test)
mse = mean_squared_error(y_test , final)
rmse = np.sqrt(mse)

print('스태킹 회귀 모델의 최종 RMSE 값은:', rmse)

In [ ]:
# 참고
# SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.
# MSSubClass: The building class
# MSZoning: The general zoning classification
# LotFrontage: Linear feet of street connected to property
# LotArea: Lot size in square feet
# Street: Type of road access
# Alley: Type of alley access
# LotShape: General shape of property
# LandContour: Flatness of the property
# Utilities: Type of utilities available
# LotConfig: Lot configuration
# LandSlope: Slope of property
# Neighborhood: Physical locations within Ames city limits
# Condition1: Proximity to main road or railroad

# Condition2: Proximity to main road or railroad (if a second is present)
# BldgType: Type of dwelling
# HouseStyle: Style of dwelling
# OverallQual: Overall material and finish quality
# OverallCond: Overall condition rating
# YearBuilt: Original construction date
# YearRemodAdd: Remodel date
# RoofStyle: Type of roof
# RoofMatl: Roof material
# Exterior1st: Exterior covering on house
# Exterior2nd: Exterior covering on house (if more than one material)
# MasVnrType: Masonry veneer type
# MasVnrArea: Masonry veneer area in square feet
# ExterQual: Exterior material quality
# ExterCond: Present condition of the material on the exterior
# Foundation: Type of foundation
# BsmtQual: Height of the basement
# BsmtCond: General condition of the basement
# BsmtExposure: Walkout or garden level basement walls
# BsmtFinType1: Quality of basement finished area
# BsmtFinSF1: Type 1 finished square feet
# BsmtFinType2: Quality of second finished area (if present)
# BsmtFinSF2: Type 2 finished square feet
# BsmtUnfSF: Unfinished square feet of basement area
# TotalBsmtSF: Total square feet of basement area
# Heating: Type of heating
# HeatingQC: Heating quality and condition
# CentralAir: Central air conditioning
# Electrical: Electrical system
# 1stFlrSF: First Floor square feet
# 2ndFlrSF: Second floor square feet
# LowQualFinSF: Low quality finished square feet (all floors)
# GrLivArea: Above grade (ground) living area square feet
# BsmtFullBath: Basement full bathrooms
# BsmtHalfBath: Basement half bathrooms
# FullBath: Full bathrooms above grade
# HalfBath: Half baths above grade
# Bedroom: Number of bedrooms above basement level
# Kitchen: Number of kitchens
# KitchenQual: Kitchen quality
# TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
# Functional: Home functionality rating
# Fireplaces: Number of fireplaces
# FireplaceQu: Fireplace quality
# GarageType: Garage location
# GarageYrBlt: Year garage was built
# GarageFinish: Interior finish of the garage
# GarageCars: Size of garage in car capacity
# GarageArea: Size of garage in square feet
# GarageQual: Garage quality
# GarageCond: Garage condition
# PavedDrive: Paved driveway
# WoodDeckSF: Wood deck area in square feet
# OpenPorchSF: Open porch area in square feet
# EnclosedPorch: Enclosed porch area in square feet
# 3SsnPorch: Three season porch area in square feet
# ScreenPorch: Screen porch area in square feet
# PoolArea: Pool area in square feet
# PoolQC: Pool quality
# Fence: Fence quality
# MiscFeature: Miscellaneous feature not covered in other categories
# MiscVal: $Value of miscellaneous feature
# MoSold: Month Sold
# YrSold: Year Sold
# SaleType: Type of sale
# SaleCondition: Condition of sale